In [1]:
# Title: Blast Wave simulation
# Author: Eric Henschel
# R = S(gamma) * E^(1/5) * (rho_o)^(-1/5)*t^(2/5)
# initial conditions::
# u(0, r) = 0 m/s
# p(0, r) = 101 kPa
# rho(0, r) = 1.225 kg/m^3


# E = 5.36*rho_o*A^2

In [140]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [79]:
# specific heat of dry air
gamma = 1.4

# initial vel
u_o = 0 # m/s

# initial pressure
p_o = 101000. # kPa

# air density
rho_o = 1.225 # kg/m^3

In [37]:
# list of nuclear weapon energy outputs in kilo-tons of TNT equivalence
# sourced: https://en.wikipedia.org/wiki/Nuclear_weapon_yield

atoms = [{'name': 'Davy Crockett', 'out': 0.02},
        {'name': 'AIR-2 Genie', 'out': 1.5},
        {'name': 'Little Boy', 'out': 18.0},
        {'name': 'Fat Man', 'out': 22.0},
        {'name': 'W76 warhead', 'out': 100.0},
        {'name': 'W87 warhead', 'out': 300.0},
        {'name': 'W88 warhead', 'out': 475.0},
        {'name': 'Ivy King', 'out': 500.0},
        {'name': 'Orange Herald Small', 'out': 720.0},
        {'name': 'B83 nuke', 'out': 1200.0},
        {'name': 'B53 nuke', 'out': 9000.0},
        {'name': 'Tsar Bomba', 'out': 50000.0},
        {'name': 'Sum_Tests', 'out': 510300.0}]

df = pd.DataFrame(atoms)
df

,name,out
0,Davy Crockett,0.02
1,AIR-2 Genie,1.50
2,Little Boy,18.00
3,Fat Man,22.00
4,W76 warhead,100.00
5,W87 warhead,300.00
6,W88 warhead,475.00
7,Ivy King,500.00
8,Orange Herald Small,720.00
9,B83 nuke,1200.00


In [42]:
# Energy output to list; names to list
names = []
energOut = []
for i, row in df.iterrows():
    out = row['out']
    name = row['name']
    energOut.append(out)
    names.append(name)

In [5]:
# Energy Kilotons TNT --> Joules
def energyConversion(E):
    return E * (4.184*10**12)

In [6]:
# radius of shock wave
def radius(t, energy):
    rho_o = 1.225 # kg/m^3
    return (1.031) * energy**(1./5) * rho_o**(-1./5) * t**(2./5)

In [7]:
# B=0.926 * (5/2) is taken from pg. 167 of
# https://www3.nd.edu/~powers/ame.90931/taylor.blast.wave.I.pdf

def velocity(radius, energy):
    rho_o = 1.225 # kg/m^3
    B = 2.31 
    return radius**(-3./2) * energy**(1/2) * (B*rho_o)**(-1./2)

In [8]:
# max pressure
def pressure(radius, energy):
    return 0.155*radius**(-3.)*energy

In [162]:
# simulating 
# radial expansion w.r.t. time
# velocity w.r.t. radius
# pressure w.r.t. radius
simulation = pd.DataFrame()
simsDf = pd.DataFrame()
radsDf = pd.DataFrame()
velsDf = pd.DataFrame()
presDf = pd.DataFrame()

for e, name in zip(energOut, names):
    # data for each Nuke
    E = energyConversion(e)
    rad = []
    vel = [u_o]
    pres = [p_o]
    time = []
    
    # Event loops
    for i in range(100):
        t = float(i)/(10**7)

        r = radius(t, E)
        rad.append(r)
        time.append(t)
        
    for r in rad[1:]:
        v = velocity(r, E)
        vel.append(v)

        p = pressure(r, E)
        pres.append(p)

    # DataFrame construction
    nameArr = np.repeat(name, len(time))
    
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    
    # dataframe for radius
    df1['name'] = nameArr
    df1['time'] = time
    df1['radius'] = rad
    display(df1)
    #pd.concat([radsDf, df1])
    
    # dataframe for velocity
    df2['name'] = nameArr
    df2['radius'] = rad
    df2['velocity'] = vel
    #pd.concat([velsDf, df2])

    # dataframe for pressure
    df3['name'] = nameArr
    df3['radius'] = rad
    df3['pressure'] = pres
    #pd.concat([presDf, df3])
    

,name,time,radius
0,Davy Crockett,0.000000e+00,0.000000
1,Davy Crockett,1.000000e-07,0.239969
2,Davy Crockett,2.000000e-07,0.316641
3,Davy Crockett,3.000000e-07,0.372395
4,Davy Crockett,4.000000e-07,0.417811
...,...,...,...
95,Davy Crockett,9.500000e-06,1.483355
96,Davy Crockett,9.600000e-06,1.489582
97,Davy Crockett,9.700000e-06,1.495769
98,Davy Crockett,9.800000e-06,1.501918


,name,time,radius
0,AIR-2 Genie,0.000000e+00,0.000000
1,AIR-2 Genie,1.000000e-07,0.569073
2,AIR-2 Genie,2.000000e-07,0.750896
3,AIR-2 Genie,3.000000e-07,0.883114
4,AIR-2 Genie,4.000000e-07,0.990814
...,...,...,...
95,AIR-2 Genie,9.500000e-06,3.517689
96,AIR-2 Genie,9.600000e-06,3.532454
97,AIR-2 Genie,9.700000e-06,3.547127
98,AIR-2 Genie,9.800000e-06,3.561709


,name,time,radius
0,Little Boy,0.000000e+00,0.000000
1,Little Boy,1.000000e-07,0.935415
2,Little Boy,2.000000e-07,1.234287
3,Little Boy,3.000000e-07,1.451619
4,Little Boy,4.000000e-07,1.628652
...,...,...,...
95,Little Boy,9.500000e-06,5.782208
96,Little Boy,9.600000e-06,5.806478
97,Little Boy,9.700000e-06,5.830596
98,Little Boy,9.800000e-06,5.854566


,name,time,radius
0,Fat Man,0.000000e+00,0.000000
1,Fat Man,1.000000e-07,0.973721
2,Fat Man,2.000000e-07,1.284832
3,Fat Man,3.000000e-07,1.511064
4,Fat Man,4.000000e-07,1.695346
...,...,...,...
95,Fat Man,9.500000e-06,6.018992
96,Fat Man,9.600000e-06,6.044255
97,Fat Man,9.700000e-06,6.069362
98,Fat Man,9.800000e-06,6.094313


,name,time,radius
0,W76 warhead,0.000000e+00,0.000000
1,W76 warhead,1.000000e-07,1.318104
2,W76 warhead,2.000000e-07,1.739249
3,W76 warhead,3.000000e-07,2.045494
4,W76 warhead,4.000000e-07,2.294953
...,...,...,...
95,W76 warhead,9.500000e-06,8.147779
96,W76 warhead,9.600000e-06,8.181977
97,W76 warhead,9.700000e-06,8.215963
98,W76 warhead,9.800000e-06,8.249739


...

,name,time,radius
0,Orange Herald Small,0.000000e+00,0.000000
1,Orange Herald Small,1.000000e-07,1.956214
2,Orange Herald Small,2.000000e-07,2.581239
3,Orange Herald Small,3.000000e-07,3.035741
4,Orange Herald Small,4.000000e-07,3.405966
...,...,...,...
95,Orange Herald Small,9.500000e-06,12.092211
96,Orange Herald Small,9.600000e-06,12.142966
97,Orange Herald Small,9.700000e-06,12.193404
98,Orange Herald Small,9.800000e-06,12.243532


,name,time,radius
0,B83 nuke,0.000000e+00,0.000000
1,B83 nuke,1.000000e-07,2.166636
2,B83 nuke,2.000000e-07,2.858894
3,B83 nuke,3.000000e-07,3.362285
4,B83 nuke,4.000000e-07,3.772333
...,...,...,...
95,B83 nuke,9.500000e-06,13.392926
96,B83 nuke,9.600000e-06,13.449140
97,B83 nuke,9.700000e-06,13.505004
98,B83 nuke,9.800000e-06,13.560524


,name,time,radius
0,B53 nuke,0.000000e+00,0.000000
1,B53 nuke,1.000000e-07,3.241890
2,B53 nuke,2.000000e-07,4.277700
3,B53 nuke,3.000000e-07,5.030913
4,B53 nuke,4.000000e-07,5.644458
...,...,...,...
95,B53 nuke,9.500000e-06,20.039539
96,B53 nuke,9.600000e-06,20.123651
97,B53 nuke,9.700000e-06,20.207239
98,B53 nuke,9.800000e-06,20.290312


,name,time,radius
0,Tsar Bomba,0.000000e+00,0.000000
1,Tsar Bomba,1.000000e-07,4.568186
2,Tsar Bomba,2.000000e-07,6.027758
3,Tsar Bomba,3.000000e-07,7.089119
4,Tsar Bomba,4.000000e-07,7.953674
...,...,...,...
95,Tsar Bomba,9.500000e-06,28.237954
96,Tsar Bomba,9.600000e-06,28.356477
97,Tsar Bomba,9.700000e-06,28.474262
98,Tsar Bomba,9.800000e-06,28.591320


,name,time,radius
0,Sum_Tests,0.000000e+00,0.000000
1,Sum_Tests,1.000000e-07,7.269673
2,Sum_Tests,2.000000e-07,9.592391
3,Sum_Tests,3.000000e-07,11.281410
4,Sum_Tests,4.000000e-07,12.657236
...,...,...,...
95,Sum_Tests,9.500000e-06,44.937027
96,Sum_Tests,9.600000e-06,45.125642
97,Sum_Tests,9.700000e-06,45.313081
98,Sum_Tests,9.800000e-06,45.499364


In [163]:
display(radsDf)

""


In [139]:
#simulation

In [116]:
# t = simulation['Sims'][0]['Time']
# r = simulation['Sims'][0]['Radius']
import plotly.express as px
df = px.data.gapminder().query("continent != 'Asia'")
display(df)

# fig = px.line(simulation)
# fig.show()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
12,Albania,Europe,1952,55.230,1282697,1601.056136,ALB,8
13,Albania,Europe,1957,59.280,1476505,1942.284244,ALB,8
14,Albania,Europe,1962,64.820,1728137,2312.888958,ALB,8
15,Albania,Europe,1967,66.220,1984060,2760.196931,ALB,8
16,Albania,Europe,1972,67.690,2263554,3313.422188,ALB,8
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716
